# Problem Statement

## Objective: 
- Minimize the total operational cost, which includes costs associated with assigning flights to time slots and aircraft, as well as possible penalties for unmet constraints.
## Constraints:
- **Aircraft availability:** Each aircraft can only be used for one flight at a time.
- **Flight schedule**
- **Crew assignment:** Each aircraft must have at least one crew


# Mathematical Formulation

**Index**:
- $i \in F $: Set of flights
- $a \in A $: Set of aircraft
- $t \in T $: Set of time slot


**Decision Variables**:
- $ x_{it} $: Binary decision variable indicating flight $i$ is assigned to timeslot $t$
- $ y_{ia} $: Binary decision variable indicating whether aircraft $a$ is used for flight $i$.

**Parameters**:
$c_{it}$: cost of assigning flight i to time slot t
$\tau_{ia}$: cost of using aircraft a for flight i
$\delta_i$: Duration of flight i
$m_a$: maintenance requirement of aircraft a
$\alpha_a$: Available slot


**Objective Function**: Minimize the total operational cost, 
which includes costs associated with assigning flights 
to time slots and aircraft
$$ \text{Minimize } \sum_{i} \sum_{t} c_{it }  x_{it} + \sum_{i} \sum_{a} \tau_{ia} y_{ia} $$

**Constraints**:
1. **Flight Assignment**: 
Each flight must be assigned exactly one time slot.
$$ \sum_{t} x_{it} = 1  \quad  \forall i$$

2. **Time Slot Capacity**:
Each time slot can accommodate at most one flight.
$$ \sum_{i} x_{it} \leq 1 \quad \forall t$$

3. **Aircraft Assignment**: 
Each flight must be assigned exactly one aircraft
$$ \sum_{a} y_{ia} = 1 \quad \forall i$$

4. **Aircraft Availability**: 
Each aircraft can handle only one flight at a time.
$$ \sum_{i} y_{ia} \leq \alpha_a \quad \forall a$$

5. **Maintenance Scheduling**: 
Ensure aircraft maintenance requirements are met.
$$ \sum_{i,t:t<=$\delta_i$} y_{ia}*\delta_i \leq m_a \quad \forall a$$

6. **Range Constraint**
$$ x_{it}, y_{ia} = \{0,1\}  $$

# Pyomo implementation

In [3]:
from pyomo.environ import ConcreteModel, Var, Constraint, Set, Binary, Objective, minimize
from pyomo.opt import SolverFactory

# Sample data
flights = ['F1', 'F2', 'F3']
time_slots = ['T1', 'T2', 'T3']
aircraft = ['A1', 'A2']
costs = {
    ('F1', 'T1'): 61000, ('F1', 'T2'): 52000, ('F1', 'T3'): 73000,
    ('F2', 'T1'): 62000, ('F2', 'T2'): 51500, ('F2', 'T3'): 72500,
    ('F3', 'T1'): 63000, ('F3', 'T2'): 52500, ('F3', 'T3'): 71000
}
aircraft_flight_costs = {
    ('A1', 'F1'): 2500, ('A1', 'F2'): 1700, ('A1', 'F3'): 3600,
    ('A2', 'F1'): 2600, ('A2', 'F2'): 1800, ('A2', 'F3'): 3500
}
flight_duration = {'F1': 2, 'F2': 3, 'F3': 1}
maintenance_req = {'A1': 5, 'A2': 5}
available_slots = {'A1': 2, 'A2': 2}

# Create a model
model = ConcreteModel()

# Define sets
model.Flights = Set(initialize=flights)
model.TimeSlots = Set(initialize=time_slots)
model.Aircraft = Set(initialize=aircraft)

# Define variables
model.x = Var(model.Flights, model.TimeSlots, within=Binary)
model.y = Var(model.Flights, model.Aircraft, within=Binary)

# Objective function
def obj_rule(model):
    return sum(costs[i, j] * model.x[i, j] for i in model.Flights for j in model.TimeSlots) + \
           sum(aircraft_flight_costs[k, i] * model.y[i, k] for i in model.Flights for k in model.Aircraft)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def flight_assignment_rule(model, i):
    return sum(model.x[i, j] for j in model.TimeSlots) == 1
model.flight_assignment = Constraint(model.Flights, rule=flight_assignment_rule)

def timeslot_assignment_rule(model, j):
    return sum(model.x[i, j] for i in model.Flights) <= 1
model.timeslot_assignment = Constraint(model.TimeSlots, rule=timeslot_assignment_rule)

def aircraft_assignment_rule(model, i):
    return sum(model.y[i, k] for k in model.Aircraft) == 1
model.aircraft_assignment = Constraint(model.Flights, rule=aircraft_assignment_rule)

def aircraft_limit_rule(model, k):
    return sum(model.y[i, k] for i in model.Flights) <= available_slots[k]
model.aircraft_limit = Constraint(model.Aircraft, rule=aircraft_limit_rule)

def maintenance_rule(model, k):
    return sum(flight_duration[i] * model.y[i, k] for i in model.Flights) <= maintenance_req[k]
model.maintenance = Constraint(model.Aircraft, rule=maintenance_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model, tee=True)

# Display results
for i in model.Flights:
    for j in model.TimeSlots:
        if model.x[i, j].value == 1:
            print(f"Flight {i} is assigned to time slot {j}")
    for k in model.Aircraft:
        if model.y[i, k].value == 1:
            print(f"Flight {i} is assigned to aircraft {k}")


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\HP\AppData\Local\Temp\tmp1_lechmb.glpk.raw --wglp C:\Users\HP\AppData\Local\Temp\tmpk_xuxsu8.glpk.glp
 --cpxlp C:\Users\HP\AppData\Local\Temp\tmp9vka_tu4.pyomo.lp
Reading problem data from 'C:\Users\HP\AppData\Local\Temp\tmp9vka_tu4.pyomo.lp'...
C:\Users\HP\AppData\Local\Temp\tmp9vka_tu4.pyomo.lp:115: warning: lower bound of variable 'x2' redefined
C:\Users\HP\AppData\Local\Temp\tmp9vka_tu4.pyomo.lp:115: warning: upper bound of variable 'x2' redefined
13 rows, 15 columns, 36 non-zeros
15 integer variables, all of which are binary
130 lines were read
Writing problem data to 'C:\Users\HP\AppData\Local\Temp\tmpk_xuxsu8.glpk.glp'...
95 lines were written
GLPK Integer Optimizer, v4.65
13 rows, 15 columns, 36 non-zeros
15 integer variables, all of which are binary
Preprocessing...
4 hidden covering inequaliti(es) were detected
13 rows, 15 columns, 36 non-zeros
15 integer variables, all of which ar